## Word Embeddings tutorial
___
Link tensorflow.org's official vector representation of words tutorial to understand the data format

In [1]:
import collections
import os
import random
import urllib
import zipfile
import numpy as np
import tensorflow as tf

In [29]:
## Hyperparameters

lr = 0.1 
bs = 128
num_steps = 300000
display_step = 10000
eval_step = 100000

# Evaluation words 
eval_words = [b'five',b'of',b'king',b'prince',b'going',b'hardware',b'electronics',b'american',b'britain']

# word2vec parameters
embedding_sz = 200
max_vocab_sz = 50000
min_occrncs = 10
skip_window = 3  # how mamy words on the left and on the right(context of total 7 words)
num_skips = 2 # How many times to reuse an input to generate a label
neg_samples = 64 # Number of negative examples to sample

In [3]:
# Download a small chunk of wikipedia articles 

url = 'http://mattmahoney.net/dc/text8.zip'
data_path = 'text8.zip'

"""
if not os.path.exists(data_path):
    print("Downloading data.....")
    filename = urllib.request.urlretrieve(url, data_path)
    print("Done!")
"""

with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()

In [4]:
# Build the dictionary and replace rare words with UNK token
count = [('UNK', -1)]

# Retrieve the most common words
count.extend(collections.Counter(text_words).most_common(max_vocab_sz-1))

# Remove samples with less than min_occrncsabs 
for i in range(len(count) -1, -1, -1):
    if(count[i][1] < min_occrncs):
        count.pop(i)
    else:
        break  ## Why break? not continue? - collection is ordered in decreasing occurences

# Compute the vocabulary size
vocab_sz = len(count)


# Assign an id to each word
word2id = dict()

for i, (word, _) in enumerate(count):
    word2id[word] = i

    
data = list()
unk_count = 0
for word in text_words:
    # Retrieve a word_id or assign it index 0 ('UNK') if not in dictionary
    index = word2id.get(word, 0)
    if(index==0):
        unk_count += 1
    data.append(index)

count[0] = ('UNK',unk_count)

id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count: ", len(text_words))
print("Unique words: ", len(set(text_words)))
print("Vocabulary size: ",vocab_sz)
print("Most Common Words: ", count[:10])


Words count:  17005207
Unique words:  253854
Vocabulary size:  47135
Most Common Words:  [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [5]:
print(data[:10])
print(text_words[:10])


[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]
[b'anarchism', b'originated', b'as', b'a', b'term', b'of', b'abuse', b'first', b'used', b'against']


In [6]:
data_index = 0

# Generate training batch for the skip-gram model
def next_batch(bs, num_skips, skip_window):
    global data_index
    assert bs%num_skips == 0
    assert num_skips <= 2*skip_window
    
    batch = np.ndarray(shape=(bs), dtype=np.int32)
    labels = np.ndarray(shape=(bs,1), dtype=np.int32)
    
    span = 2*skip_window + 1 # (left3, left2, left1, current, right1, right2, right3) in the context
    buffer = collections.deque(maxlen=span)
    
    if data_index + span > len(data):
        data_index=0
    
    buffer.extend(data[data_index:data_index+span])
    data_index += span
    
    for i in range(bs//num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_2_use = random.sample(context_words, num_skips)
        
        for j, context_word in enumerate(words_2_use):
            batch[i*num_skips + j] = buffer[skip_window]
            labels[i*num_skips + j, 0] = buffer[context_word]
            
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data[data_index]])
            data_index += 1
    data_index = (data_index + len(data) - span)%len(data)
    return batch, labels

In [42]:
## Define the architecture

X = tf.placeholder(tf.int32, shape=[None])
Y = tf.placeholder(tf.int32, shape=[None, 1])

# Embedding matrix
embedding = tf.Variable(tf.random_normal([vocab_sz, embedding_sz]))


# Embedding lookup
X_embed = tf.nn.embedding_lookup(embedding, X)

# Construct Variables for the NCE loss
nce_weights = tf.Variable(tf.random_normal([vocab_sz,embedding_sz]))
nce_biases = tf.Variable(tf.zeros([vocab_sz]))

# Compute the average NCE Loss for the batch
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, 
                                     biases=nce_biases,
                                     labels=Y,
                                     inputs=X_embed,
                                     num_sampled=neg_samples,
                                     num_classes=vocab_sz))

print(loss)
# Define the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)
print(train_op)


# Evalutation
# Compute the cosine similarity between the input data embedding and every embedding vectors
X_embed_norm = X_embed/tf.sqrt(tf.reduce_sum(tf.square(X_embed)))
embed_norm = embedding/tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
cosine_sim_op = tf.matmul(X_embed_norm, embed_norm, transpose_b=True)


Tensor("Mean_5:0", shape=(), dtype=float32)
name: "GradientDescent_4"
op: "NoOp"
input: "^GradientDescent_4/update_Variable_21/ScatterSub"
input: "^GradientDescent_4/update_Variable_22/ScatterSub"
input: "^GradientDescent_4/update_Variable_23/ScatterSub"



In [ ]:
## Execute the graph
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)   # Edges in the graphs are initialized
    
    x_test = np.array([word2id[w] for w in eval_words])
    avg_loss = 0
    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(bs, num_skips, skip_window)
#         print(batch_x, batch_y)
        # train
        _, avg_loss = sess.run([train_op, loss], feed_dict={X:batch_x, Y:batch_y})
#         avg_loss = loss
        
        if(step%display_step)==0:
            print("Step " + str(step) + ", Average Loss= " + str(avg_loss))
        
        #Evaluation
        if(step%eval_step)==0:
            print("Evaluation ...")
            sim = sess.run(cosine_sim_op, feed_dict={X:x_test})
            for i in range(len(eval_words)):
                top_k = 8    # number of nearest words
                nearest = (-sim[i,:]).argsort()[1:top_k + 1]
                log_str = str(eval_words[i]) + " nearest neighbours: "
                for k in range(top_k):
                    log_str += str(id2word[nearest[k]])
                print(log_str)

Step 10000, Average Loss= 52.663338
Step 20000, Average Loss= 23.424046
Step 30000, Average Loss= 28.799204
Step 40000, Average Loss= 7.9404426
Step 50000, Average Loss= 18.83305
Step 60000, Average Loss= 10.993977
Step 70000, Average Loss= 17.386383
Step 80000, Average Loss= 13.522605
Step 90000, Average Loss= 9.814818
Step 100000, Average Loss= 9.065243
Evaluation ...
b'five' nearest neighbours: b'nine'b'eight'b'zero'b'two'b'three'b'one'b'four'b'he'
b'of' nearest neighbours: b'the'b'in'b'and'b'as'UNKb'is'b'anarchists'b'to'
b'king' nearest neighbours: b'at'b'his'b'political'b'what'b'are'b'was'b'with'b's'
b'prince' nearest neighbours: b'temperature'b'aquaculture'b'carriages'b'whatever'b'maintaining'b'relevant'b'failure'b'better'
b'going' nearest neighbours: b'organisation'b'active'b'de'b'set'b'europe'b'should'b'times'b'visions'
b'hardware' nearest neighbours: b'roms'b'zangger'b'inlay'b'ballarat'b'pp'b'torment'b'hillary'b'electrophilic'
b'electronics' nearest neighbours: b'meeting'b'int

In [31]:
[print(i) for i in eval_words]

b'five'
b'of'
b'king'
b'prince'
b'going'
b'hardware'
b'electronics'
b'american'
b'britain'


[None, None, None, None, None, None, None, None, None]